In [25]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import math
%matplotlib inline

### chi-squared goodness of fit

t检验判断样本均值是否有总体均值不同，卡方goodness-of-fit 是相当与categorical变量的t检验，它检验categorical数据样本分布与期望分布是否一致

$$sum(\frac{(observed-expected)^2}{expected})$$

In [26]:
national = pd.DataFrame(["white"]*100000 + ["hispanic"]*60000 + ["black"]*50000 + ["asian"]*15000 + ["other"]*35000)
minnesota = pd.DataFrame(["white"]*600 + ["hispanic"]*300 + ["black"]*250 +["asian"]*75 + ["other"]*150)
national_table = pd.crosstab(index=national[0], columns="count")
minnesota_table = pd.crosstab(index=minnesota[0], columns="count")
print("National")
print(national_table)
print(" ")
print( "Minnesota")
print(minnesota_table)

National
col_0      count
0               
asian      15000
black      50000
hispanic   60000
other      35000
white     100000
 
Minnesota
col_0     count
0              
asian        75
black       250
hispanic    300
other       150
white       600


In [27]:
observed = minnesota_table
national_ratios = national_table/len(national) # population ratios 
expected = national_ratios * len(minnesota) # expected counts 
chi_squared_stat = (((observed-expected)**2)/expected).sum()
print(chi_squared_stat)

col_0
count    18.194805
dtype: float64


In [28]:
crit = stats.chi2.ppf(q = 0.95, #95% confidence
                      df = 4) # Df = number of variable categories - 1 
print("Critical value")
print(crit)
p_value = 1 - stats.chi2.cdf(x=chi_squared_stat, # p-value 
                             df=4)
print("P value")
print(p_value)

Critical value
9.48772903678
P value
[ 0.00113047]


In [29]:
stats.chisquare(f_obs= observed, # Array of observed counts 
                f_exp= expected) # Array of expected counts

Power_divergenceResult(statistic=array([ 18.19480519]), pvalue=array([ 0.00113047]))

### chi-square independence test

独立性是指知道一个变量不会告诉你任何关于另一个变量的信息。比如你是几月出生的不会决定你的工资水平。卡方独立性检验就是用于判断两个分类变量是否独立。

In [39]:
np.random.seed(2016) 
# race
voter_race = np.random.choice(a= ["asian","black","hispanic","other","white"], 
                              p = [0.05, 0.15 ,0.25, 0.05, 0.5], size=1000) 
# party
voter_party = np.random.choice(a= ["democrat","independent","republican"], 
                               p = [0.4, 0.2, 0.4], size=1000) 
voters = pd.DataFrame({"race":voter_race, "party":voter_party}) 
voter_tab = pd.crosstab(voters.race, voters.party, margins = True) 
voter_tab.columns = ["democrat","independent","republican","row_totals"] 
voter_tab.index = ["asian","black","hispanic","other","white","col_totals"] 
observed = voter_tab.ix[0:5,0:3] # no totals
voter_tab

,democrat,independent,republican,row_totals
asian,18,4,16,38
black,56,36,58,150
hispanic,100,61,96,257
other,29,11,32,72
white,197,94,192,483
col_totals,400,206,394,1000


In [40]:
voter_tab["row_totals"][0:5]

asian        38
black       150
hispanic    257
other        72
white       483
Name: row_totals, dtype: int64

In [41]:
voter_tab.ix["col_totals"][0:3]

democrat       400
independent    206
republican     394
Name: col_totals, dtype: int64

In [14]:
np.outer(voter_tab["row_totals"][0:5], 
                    voter_tab.ix["col_totals"][0:3])

array([[ 15200,   7828,  14972],
       [ 60000,  30900,  59100],
       [102800,  52942, 101258],
       [ 28800,  14832,  28368],
       [193200,  99498, 190302]], dtype=int64)

In [42]:
expected = np.outer(voter_tab["row_totals"][0:5], 
                    voter_tab.ix["col_totals"][0:3]) / 1000 
expected = pd.DataFrame(expected)
expected.columns = ["democrat","independent","republican"] 
expected.index = ["asian","black","hispanic","other","white"] 
expected

,democrat,independent,republican
asian,15.2,7.828,14.972
black,60.0,30.900,59.100
hispanic,102.8,52.942,101.258
other,28.8,14.832,28.368
white,193.2,99.498,190.302


In [43]:
chi_squared_stat = (((observed-expected)**2)/expected).sum().sum() 
print(chi_squared_stat)

7.01309736929


In [44]:
crit = stats.chi2.ppf(q = 0.95, # critical value for 95% confidence* 
                      df = 8) #(5-1)*(3-1) 
print("Critical value") 
print(crit)
# p-value 
p_value = 1 - stats.chi2.cdf(x=chi_squared_stat, 
                                         df=8) 
print("P value") 
print(p_value)

Critical value
15.5073130559
P value
0.535220220178


In [20]:
stats.chi2_contingency(observed=observed)

(7.0130973692853118,
 0.53522022017829296,
 8,
 array([[  15.2  ,    7.828,   14.972],
        [  60.   ,   30.9  ,   59.1  ],
        [ 102.8  ,   52.942,  101.258],
        [  28.8  ,   14.832,   28.368],
        [ 193.2  ,   99.498,  190.302]]))

chi-square statistic, p-value ,degrees of freedom,expected counts.